# Precision and Recall

[![8DS6l8.png](https://s1.ax1x.com/2020/03/18/8DS6l8.png)](https://imgchr.com/i/8DS6l8)
[![8DSaeH.png](https://s1.ax1x.com/2020/03/18/8DSaeH.png)](https://imgchr.com/i/8DSaeH)
[![8DSwTA.png](https://s1.ax1x.com/2020/03/18/8DSwTA.png)](https://imgchr.com/i/8DSwTA)

### 1. 实现混淆矩阵，精准率和召回率

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()

X = digits.data
y = digits.target.copy()

# 把十个数字分类问题转化成二分类问题. 把数据集变成一个极度偏斜的数据集
y[digits.target==9] = 1
y[digits.target!=9] = 0

In [3]:
np.sum(y == 1)

180

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [5]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(multi_class='auto')
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

D:\appCommon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9755555555555555

In [6]:
y_log_predict = log_reg.predict(X_test)

In [7]:
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true==0) & (y_predict==0))
TN(y_test, y_log_predict)

403

In [8]:
def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true==1) & (y_predict==0))
FN(y_test, y_log_predict)

9

In [9]:
def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true==0) & (y_predict==1))

FP(y_test, y_log_predict)

2

In [10]:
def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true==1) & (y_predict==1))

TP(y_test, y_log_predict)

36

In [11]:
def confusion_matrix(y_true, y_predict):
    return np.array([
        [TN(y_true, y_predict), FP(y_true, y_predict)],
        [FN(y_true, y_predict), TP(y_true, y_predict)]
    ])

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [12]:
def precision_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fp = FP(y_true, y_predict)
    try:
        return tp / (tp + fp)
    except:
        return 0.0
    
precision_score(y_test, y_log_predict)

0.9473684210526315

In [13]:
def recall_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp / (tp + fn)
    except:
        return 0.0
    
recall_score(y_test, y_log_predict)

0.8

### 2. scikit-learn 中的混淆矩阵，精准率和召回率

In [14]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]], dtype=int64)

In [15]:
from sklearn.metrics import precision_score

precision_score(y_test, y_log_predict)

0.9473684210526315

In [16]:
from sklearn.metrics import recall_score

recall_score(y_test, y_log_predict)

0.8

### 3. 如何取舍

- 在有些领域，如预测股票，比较看重精准率
- 而在有些领域，如病症预测，比较看重召回率

- 如果想要兼顾二者，可以使用 F1 Score
      - F1 Score 是 precision 和 recall 的 调和平均值

$$\frac{1}{F1} = \frac{1}{2}(\frac{1}{precision} + \frac{1}{recall})$$
$$\Downarrow$$
$$F1 = \frac{2\cdot precision \cdot recall}{precision + recall}$$

### 4. F1 Score

In [17]:
def f1_score(precision, recall):
    try:
        return (2 * precision * recall) / (precision + recall)
    except:
        return 0.0

In [18]:
precision = 0.5
recall = 0.5
f1_score(precision, recall)

0.5

In [19]:
precision = 0.9
recall = 0.1
f1_score(precision, recall)

0.18000000000000002

In [20]:
precision = 0.0
recall = 1
f1_score(precision, recall)

0.0

In [21]:
confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]], dtype=int64)

In [22]:
precision_score(y_test, y_log_predict)

0.9473684210526315

In [23]:
recall_score(y_test, y_log_predict)

0.8

#### 使用 scikit-learn中的 F1 Score 

In [24]:
from sklearn.metrics import f1_score

In [25]:
f1_score(y_test, y_log_predict)

0.8674698795180723

### 5. 精准率和召回率之间的平衡

[![8DSySf.png](https://s1.ax1x.com/2020/03/18/8DSySf.png)](https://imgchr.com/i/8DSySf)

- 逻辑回归默认是以 0 为基准来分类的

In [26]:
# 可以用 decision_function 查看每一个样本的的 score 值是多少
log_reg.decision_function(X_test)[:10]

array([-22.05700117, -33.02940957, -16.21334087, -80.3791447 ,
       -48.25125396, -24.54005629, -44.39168773, -25.04292757,
        -0.97829292, -19.7174399 ])

In [27]:
log_reg.predict(X_test)[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [28]:
decision_score = log_reg.decision_function(X_test)

In [29]:
np.min(decision_score)

-85.68608522646575

In [30]:
np.max(decision_score)

19.8895858799022

##### 我们以 10 为基准来分类 

In [31]:
y_predict_2 = np.array(decision_score >= 10, dtype='int')

In [32]:
confusion_matrix(y_test, y_predict_2)

array([[405,   0],
       [ 32,  13]], dtype=int64)

In [33]:
precision_score(y_test, y_predict_2)

1.0

In [34]:
recall_score(y_test, y_predict_2)

0.28888888888888886

- 基准为 10 时，与基准为 0 相比，精准率上升，召回率降低